### Player Analytics

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os

#### Load Data

In [2]:
player_game_stats = pd.read_parquet('../data/player_game_stats.parquet')
all_games = pd.read_parquet('../data/all_games.parquet')

In [3]:
player_game_stats.loc[player_game_stats.name.str.contains('Nikola') == True].head()

,status,personId,jerseyNum,position,starter,played,name,GAME_ID,assists,blocks,...,reboundsTotal,steals,threePointersAttempted,threePointersMade,threePointersPercentage,turnovers,twoPointersAttempted,twoPointersMade,twoPointersPercentage,TEAM_SIDE
2,ACTIVE,203999,15,C,1,1,Nikola Jokić,0022400461,15,1,...,17,0,1,1,1.0,2,15,7,0.466667,homeTeam
5,ACTIVE,1631107,5,None,0,1,Nikola Jović,0022400427,1,0,...,2,0,2,0,0.0,1,1,0,0.000000,awayTeam
2,ACTIVE,202696,9,C,1,1,Nikola Vučević,0022400413,3,0,...,8,2,1,1,1.0,0,12,8,0.666667,awayTeam
2,ACTIVE,203999,15,C,1,1,Nikola Jokić,0022400350,8,0,...,14,3,6,3,0.5,3,23,14,0.608696,awayTeam
2,ACTIVE,202696,9,C,1,1,Nikola Vučević,0022400030,3,4,...,13,1,5,2,0.4,4,9,5,0.555556,homeTeam


In [12]:
# Nuggets Regular Season

nuggets_games_24 = all_games.loc[(all_games.TEAM == 'DEN') & (all_games.SEASON_SHORT == 2024) & (all_games.GAME_TYPE == 'regular')]
nuggets_games_24_short = nuggets_games_24[['TEAM', 'GAME_ID', 'GAME_DATE', 'TEAM_SIDE', 'OPPONENT', 'WL', 'SEASON_SHORT', 'GAME_TYPE']]

nuggets_GAME_ID_24 = list(nuggets_games_24.GAME_ID)

print(nuggets_games_24_short.shape)
nuggets_games_24_short.head()

(35, 8)


,TEAM,GAME_ID,GAME_DATE,TEAM_SIDE,OPPONENT,WL,SEASON_SHORT,GAME_TYPE
0,DEN,0022400507,2025-01-07,homeTeam,BOS,L,2024,regular
1,DEN,0022400484,2025-01-04,awayTeam,SAS,W,2024,regular
2,DEN,0022400475,2025-01-03,homeTeam,SAS,L,2024,regular
3,DEN,0022400461,2025-01-01,homeTeam,ATL,W,2024,regular
4,DEN,0022400445,2024-12-30,awayTeam,UTA,W,2024,regular


In [16]:
nugg_player_game_stats = pd.merge(player_game_stats, nuggets_games_24_short, on=['GAME_ID'])
nugg_player_game_stats = nugg_player_game_stats.loc[nugg_player_game_stats.TEAM_SIDE_x == nugg_player_game_stats.TEAM_SIDE_y] \
    .drop('TEAM_SIDE_y', axis = 1).rename(columns={'TEAM_SIDE_x': 'TEAM_SIDE'})

nugg_player_game_stats.head()

,status,personId,jerseyNum,position,starter,played,name,GAME_ID,assists,blocks,...,twoPointersAttempted,twoPointersMade,twoPointersPercentage,TEAM_SIDE,TEAM,GAME_DATE,OPPONENT,WL,SEASON_SHORT,GAME_TYPE
0,ACTIVE,1631128,0,SF,1,1,Christian Braun,0022400461,4,0,...,8,7,0.875000,homeTeam,DEN,2025-01-01,ATL,W,2024,regular
1,ACTIVE,1629008,1,PF,1,1,Michael Porter Jr.,0022400461,2,0,...,5,3,0.600000,homeTeam,DEN,2025-01-01,ATL,W,2024,regular
2,ACTIVE,203999,15,C,1,1,Nikola Jokić,0022400461,15,1,...,15,7,0.466667,homeTeam,DEN,2025-01-01,ATL,W,2024,regular
3,ACTIVE,201566,4,SG,1,1,Russell Westbrook,0022400461,11,0,...,4,4,1.000000,homeTeam,DEN,2025-01-01,ATL,W,2024,regular
4,ACTIVE,1627750,27,PG,1,1,Jamal Murray,0022400461,2,1,...,9,4,0.444444,homeTeam,DEN,2025-01-01,ATL,W,2024,regular


In [40]:
nugg_player_stats_agg = nugg_player_game_stats.loc[nugg_player_game_stats.status == 'ACTIVE'][['personId', 'name', 'fieldGoalsAttempted','fieldGoalsMade', 'threePointersAttempted', 'threePointersMade', 'freeThrowsAttempted',
       'freeThrowsMade', 'points', 'reboundsTotal', 'assists', 'blocks']].groupby(['personId', 'name']).agg(['mean', 'std'])

nugg_player_stats_agg.head()

fieldGoalsAttempted           fieldGoalsMade  \
                                          mean       std           mean   
personId name                                                             
201566   Russell Westbrook           10.085714  3.118931       4.600000   
201599   DeAndre Jordan               1.457143  1.668794       0.885714   
203932   Aaron Gordon                 9.117647  3.854905       4.764706   
203967   Dario Šarić                  1.400000  2.607681       0.400000   
203999   Nikola Jokić                21.677419  6.896313      12.000000   

                                     threePointersAttempted            \
                                 std                   mean       std   
personId name                                                           
201566   Russell Westbrook  2.475765               3.600000  1.752309   
201599   DeAndre Jordan     1.078436               0.000000  0.000000   
203932   Aaron Gordon       2.862023               2.941176  1.434860   
203967   Dario Šarić        1.069966               0.666667  1.446359   
203999   Nikola Jokić       3.642344               4.806452  2.749780   

                           threePointersMade           freeThrowsAttempted  \
                                        mean       std                mean   
personId name                                                                
201566   Russell Westbrook          1.171429  1.124218            3.228571   
201599   DeAndre Jordan             0.000000  0.000000            0.485714   
203932   Aaron Gordon               1.294118  1.311712            3.823529   
203967   Dario Šarić                0.166667  0.530669            0.200000   
203999   Nikola Jokić               2.290323  1.442071            6.483871   

                                     freeThrowsMade               points  \
                                 std           mean       std       mean   
personId name                                                              
201566   Russell Westbrook  2.755590       2.057143  2.400280  12.428571   
201599   DeAndre Jordan     1.094678       0.171429  0.452816   1.942857   
203932   Aaron Gordon       2.455486       2.882353  2.147160  13.705882   
203967   Dario Šarić        0.610257       0.166667  0.530669   1.133333   
203999   Nikola Jokić       3.443554       5.225806  2.940631  31.516129   

                                     reboundsTotal             assists  \
                                 std          mean       std      mean   
personId name                                                            
201566   Russell Westbrook  6.971732      4.771429  2.981173  6.628571   
201599   DeAndre Jordan     2.222120      2.657143  2.817413  0.571429   
203932   Aaron Gordon       7.662936      5.764706  3.326586  3.058824   
203967   Dario Šarić        2.775923      1.200000  2.426861  0.500000   
203999   Nikola Jokić       9.146478     13.032258  4.658210  9.709677   

                                        blocks            
                                 std      mean       std  
personId name                                             
201566   Russell Westbrook  3.000840  0.542857  0.610827  
201599   DeAndre Jordan     0.884032  0.285714  0.518563  
203932   Aaron Gordon       1.919329  0.176471  0.392953  
203967   Dario Šarić        1.252584  0.033333  0.182574  
203999   Nikola Jokić       3.866036  0.612903  0.715422

In [41]:
nugg_player_stats_agg['points']['mean'] / nugg_player_stats_agg['points']['std']

personId  name              
201566    Russell Westbrook     1.782709
201599    DeAndre Jordan        0.874326
203932    Aaron Gordon          1.788594
203967    Dario Šarić           0.408273
203999    Nikola Jokić          3.445712
1627750   Jamal Murray          3.071238
1628427   Vlatko Čančar         0.680336
1629008   Michael Porter Jr.    3.601796
1629618   Jalen Pickett         0.412837
1630192   Zeke Nnaji            0.503632
1631124   Julian Strawther      1.910550
1631128   Christian Braun       3.161415
1631212   Peyton Watson         1.692087
1641725   Trey Alexander        0.450483
1641790   PJ Hall               0.393398
1641816   Hunter Tyson          0.679027
1642461   Spencer Jones         0.250000
dtype: float64

In [38]:
# All Regular Season

all_games_24 = all_games.loc[(all_games.SEASON_SHORT == 2024) & (all_games.GAME_TYPE == 'regular')]
all_games_24_short = all_games_24[['TEAM', 'GAME_ID', 'GAME_DATE', 'TEAM_SIDE', 'OPPONENT', 'WL', 'SEASON_SHORT', 'GAME_TYPE']]

all_GAME_ID_24 = list(all_games_24.GAME_ID)

print(all_games_24_short.shape)
all_games_24_short.head()

(1076, 8)


,TEAM,GAME_ID,GAME_DATE,TEAM_SIDE,OPPONENT,WL,SEASON_SHORT,GAME_TYPE
0,ATL,0022400506,2025-01-07,awayTeam,UTA,W,2024,regular
1,ATL,0022400486,2025-01-04,awayTeam,LAC,L,2024,regular
2,ATL,0022400477,2025-01-03,awayTeam,LAL,L,2024,regular
3,ATL,0022400461,2025-01-01,awayTeam,DEN,L,2024,regular
4,ATL,0022400438,2024-12-29,awayTeam,TOR,W,2024,regular


In [39]:
all_player_game_stats = pd.merge(player_game_stats, all_games_24_short, on=['GAME_ID'])
all_player_game_stats = all_player_game_stats.loc[all_player_game_stats.TEAM_SIDE_x == all_player_game_stats.TEAM_SIDE_y] \
    .drop('TEAM_SIDE_y', axis = 1).rename(columns={'TEAM_SIDE_x': 'TEAM_SIDE'})

all_player_game_stats.head()

,status,personId,jerseyNum,position,starter,played,name,GAME_ID,assists,blocks,...,twoPointersAttempted,twoPointersMade,twoPointersPercentage,TEAM_SIDE,TEAM,GAME_DATE,OPPONENT,WL,SEASON_SHORT,GAME_TYPE
1,ACTIVE,1630548,33,SF,1,1,Johnny Juzang,0022400506,4,0,...,3,1,0.333333,homeTeam,UTA,2025-01-07,ATL,L,2024,regular
3,ACTIVE,1628374,23,PF,1,1,Lauri Markkanen,0022400506,1,1,...,6,3,0.500000,homeTeam,UTA,2025-01-07,ATL,L,2024,regular
5,ACTIVE,1631117,24,C,1,1,Walker Kessler,0022400506,0,3,...,11,8,0.727273,homeTeam,UTA,2025-01-07,ATL,L,2024,regular
7,ACTIVE,1629012,2,SG,1,1,Collin Sexton,0022400506,3,0,...,13,5,0.384615,homeTeam,UTA,2025-01-07,ATL,L,2024,regular
9,ACTIVE,1642268,13,PG,1,1,Isaiah Collier,0022400506,9,0,...,2,0,0.000000,homeTeam,UTA,2025-01-07,ATL,L,2024,regular


In [42]:
all_player_stats_agg = all_player_game_stats.loc[all_player_game_stats.status == 'ACTIVE'][['personId', 'name', 'fieldGoalsAttempted','fieldGoalsMade', 'threePointersAttempted', 'threePointersMade', 'freeThrowsAttempted',
       'freeThrowsMade', 'points', 'reboundsTotal', 'assists', 'blocks']].groupby(['personId', 'name']).agg(['mean', 'std'])

all_player_stats_agg.head()

fieldGoalsAttempted           fieldGoalsMade            \
                                     mean       std           mean       std   
personId name                                                                  
2544     LeBron James           18.060606  4.227436       9.151515  3.173338   
101108   Chris Paul              7.722222  2.536715       3.166667  1.919821   
200768   Kyle Lowry              3.222222  2.342473       1.148148  1.406132   
201142   Kevin Durant           18.640000  4.554119       9.640000  2.447448   
201143   Al Horford              6.382353  3.798043       2.705882  2.052780   

                      threePointersAttempted           threePointersMade  \
                                        mean       std              mean   
personId name                                                              
2544     LeBron James               5.727273  2.503407          2.212121   
101108   Chris Paul                 4.777778  1.623244          1.638889   
200768   Kyle Lowry                 2.407407  1.845145          0.814815   
201142   Kevin Durant               5.680000  2.174090          2.320000   
201143   Al Horford                 4.647059  3.151549          1.705882   

                                freeThrowsAttempted           freeThrowsMade  \
                            std                mean       std           mean   
personId name                                                                  
2544     LeBron James  1.916337            4.333333  2.445233       3.333333   
101108   Chris Paul    1.125110            1.444444  1.538913       1.416667   
200768   Kyle Lowry    1.110684            0.925926  1.591466       0.740741   
201142   Kevin Durant  1.029563            6.840000  3.261901       5.760000   
201143   Al Horford    1.605472            0.352941  0.733706       0.294118   

                                    points           reboundsTotal            \
                            std       mean       std          mean       std   
personId name                                                                  
2544     LeBron James  1.947220  23.848485  8.023875      7.666667  3.406489   
101108   Chris Paul    1.537623   9.388889  4.888925      4.305556  2.435681   
200768   Kyle Lowry    1.318291   3.851852  4.417654      1.703704  1.353480   
201142   Kevin Durant  3.179098  27.360000  6.563028      6.520000  2.238303   
201143   Al Horford    0.629064   7.411765  5.862659      4.676471  2.590432   

                        assists              blocks            
                           mean       std      mean       std  
personId name                                                  
2544     LeBron James  8.848485  3.083436  0.545455  0.904534  
101108   Chris Paul    8.361111  2.860098  0.194444  0.467177  
200768   Kyle Lowry    2.518519  2.045076  0.296296  0.465322  
201142   Kevin Durant  3.920000  2.253146  1.280000  1.208305  
201143   Al Horford    1.882353  1.552412  0.705882  0.871412

In [47]:
all_player_points_agg = all_player_stats_agg['points']
all_player_points_agg['ratio'] =  all_player_stats_agg['points']['mean'] /  all_player_stats_agg['points']['std']

all_player_points_agg.sort_values(by='ratio', ascending = False).head(20)

/tmp/ipykernel_40117/1187050480.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_player_points_agg['ratio'] =  all_player_stats_agg['points']['mean'] /  all_player_stats_agg['points']['std']


,,mean,std,ratio
personId,name,,,
1628983,Shai Gilgeous-Alexander,31.264706,6.524122,4.792170
201142,Kevin Durant,27.360000,6.563028,4.168808
1627734,Domantas Sabonis,20.588235,5.003742,4.114568
1627783,Pascal Siakam,19.648649,4.950961,3.968654
1628381,John Collins,17.925926,4.937501,3.630566
1629008,Michael Porter Jr.,19.028571,5.283079,3.601796
1629639,Tyler Herro,23.333333,6.529056,3.573768
202695,Kawhi Leonard,10.000000,2.828427,3.535534
1630163,LaMelo Ball,29.916667,8.677390,3.447657


In [51]:
all_player_points_agg.loc[all_player_points_agg['mean'] > 25].sort_values(by='ratio')

,,mean,std,ratio
personId,name,,,
1631094,Paolo Banchero,29.000000,14.370108,2.018078
203076,Anthony Davis,25.085714,10.931260,2.294860
1641705,Victor Wembanyama,25.612903,9.618307,2.662933
1628369,Jayson Tatum,26.583333,9.466859,2.808041
203507,Giannis Antetokounmpo,30.428571,10.785548,2.821236
1630178,Tyrese Maxey,25.392857,8.655289,2.933797
1628368,De'Aaron Fox,26.657143,8.663456,3.076964
1630162,Anthony Edwards,25.805556,8.335190,3.095977
1629029,Luka Dončić,28.136364,8.362589,3.364552
